In [21]:
import requests # 
import shutil  
import json

from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
import time

import os


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



C:\Users\suuser\AppData\Roaming\Python\Python311\site-packages\IPython\core\completerlib.py:146: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


In [4]:
with open("amazon_uk_shoes_dataset.json", "r") as fh:
    amazon_data = json.load(fh)

In [5]:
all_image_counts = 0
for i in amazon_data:
    image_urls = i.get("images_list", [])
    all_image_counts += len(image_urls)

In [6]:
all_image_counts

70352

In [7]:
i

{'url': 'https://www.amazon.co.uk/dp/B07TD54MBQ',
 'title': 'viking Jolly Winter Rain Boot Child',
 'asin': 'B07TD54MBQ',
 'price': '£18.32 - £43.87',
 'brand': 'Visit the Viking Store',
 'product_details': 'Package Dimensions\n\u200f\n:\n\u200e\n\n20.2 x 17.4 x 11 cm; 562 Grams\n\n\nDate First Available\n\u200f\n:\n\u200e\n\n30 Jun. 2019\n\n\nManufacturer\n\u200f\n:\n\u200e\n\nViking\n\n\nASIN\n\u200f\n:\n\u200e\n\nB07TGD1QQH\n\n\nItem model number\n\u200f\n:\n\u200e\n\n1-12150\n\n\nDepartment\n\u200f\n:\n\u200e\n\nUnisex Kids',
 'breadcrumbs': "Shoes/Boys' Shoes/Boots",
 'images_list': ['https://m.media-amazon.com/images/I/31LftlwGa6L.jpg',
  'https://m.media-amazon.com/images/I/314OUUenRxL.jpg',
  'https://m.media-amazon.com/images/I/31ZCevc9vOL.jpg',
  'https://m.media-amazon.com/images/I/31xIGtCPMdL.jpg',
  'https://m.media-amazon.com/images/I/31o6E9QKrVL.jpg',
  'https://m.media-amazon.com/images/I/31TkWx7zo6L.jpg',
  'https://m.media-amazon.com/images/I/41cGs+EwDDL.jpg'],
 'feat

In [26]:
urls_ids = []
for i in amazon_data:
    image_urls = i.get("images_list", [])
    for index, img_url in enumerate(image_urls):
        urls_ids.append((i["asin"], img_url, index))

In [18]:
def download_image(url_id_index):
    product_id, image_url, index = url_id_index
    res = requests.get(image_url, stream=True)
    if res.status_code == 200:
        file_name = product_id + "_" + str(index)
        with open(f"images/{file_name}.jpg" ,'wb') as f:
            shutil.copyfileobj(res.raw, f)
            time.sleep(0.1)
    else:
        with open("errors.txt", "a") as ef:
            ef.write(product_id + "\n")

In [29]:
with ThreadPool(10) as pool:
    with tqdm(total=len(urls_ids), desc='Downloading Images') as progress_bar:
        for _ in pool.imap_unordered(download_image, urls_ids):
            progress_bar.update(1)